## Tracking Corona on your Smartphone Using Python - Tutorial

This notebook accompanies the [Tracking Corona on your Smartphone Using Python Medium Post]().  
We will go through each part of the code step by step.  

The First Step is to **Import all the Libraries**.  

We are going to use the [Selenium package](https://www.selenium.dev/) to extract data from the [Worldometers website](https://www.worldometers.info/coronavirus/). This process is called Web-Scraping. If you have never heard this term before, I would recommend going through this [tutorial](https://towardsdatascience.com/web-scraping-using-selenium-python-8a60f4cf40ab).

In [1]:
# Scraping library
from selenium import webdriver

# Mail Libraries
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Table Library
from tabulate import tabulate


The next step is to **setup the driver** and load the required URL

In [2]:
# setting up driver and loading URL
driver = webdriver.Chrome('path/to/chromedriver')
driver.get('https://www.worldometers.info/coronavirus/')

Now, we will ask user for the country whose data they want. Then we will extract the table on Worldometers website that provides information about confirmed cases and deaths in each country.

In [4]:
Country_name = input() # asking user for country

# getting table out
table = driver.find_element_by_xpath('//*[@id="main_table_countries_today"]/tbody[1]')

# getting country row out
xpath = "//td[contains(text(), '%s')]"% Country_name
country_element = table.find_element_by_xpath(xpath)
row = country_element.find_element_by_xpath("./..")

Netherlands


Now, since we have a pointer to the required row, we will extract data from each cell in the row. All values are then stored in a list.

In [5]:
# extracting data from each row_cell
col_data = row.find_elements_by_tag_name("td")
data_val = [x.text for x in col_data]
print(data_val)

['Netherlands', '2,460', '+409', '76', '+18', '2', '2,382', '45', '144']


We will first convert this list into a dict with proper labels. Then, we will use the tabulate library and convert into a table.

In [6]:
# converting data into a dictionary and then a table
data_keys = ['Country', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths','Active Cases','Total Recovered','Serious, critical cases','Total Cases per million']
data_dict = dict(zip(data_keys,data_val))
table=tabulate(data_dict.items(), tablefmt="grid")

print(table)

+-------------------------+-------------+
| Country                 | Netherlands |
+-------------------------+-------------+
| Total Cases             | 2,460       |
+-------------------------+-------------+
| New Cases               | +409        |
+-------------------------+-------------+
| Total Deaths            | 76          |
+-------------------------+-------------+
| New Deaths              | +18         |
+-------------------------+-------------+
| Active Cases            | 2           |
+-------------------------+-------------+
| Total Recovered         | 2,382       |
+-------------------------+-------------+
| Serious, critical cases | 45          |
+-------------------------+-------------+
| Total Cases per million | 144         |
+-------------------------+-------------+


Now that we have the extracted data in a python dictionary, we can send this as an **email update** every morning.  

Firstly, you need to login to your Gmail account and go to App Passwords.  

Here, set up a new app and generate a new password. Use it in server.login() in the code below. The script might seem a little complicated. This is  only because I wanted the data to be formatted as a table.

In [7]:
def send_mail_table(data_dict):
    # setting mail up
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    
    # setting login details
    server.login('email_addr', 'app_password')
    
    # mail subject
    subject = 'COVID-19 stats for your country'
    
    # mail body
    text = """Data regarding COVID-19 in your country today. Source: https://www.worldometers.info/coronavirus/ 
    
    {table}

    Regards,

    Covid-update"""
    
    # mail html
    html = """
    <html>
    <head>
    <style> 
      table, th, td {{ border: 1px solid black; border-collapse: collapse; }}
      th, td {{ padding: 5px; }}
    </style>
    </head>
    <body><p>Data regarding COVID-19 in your country today. Source: 
    <a href="https://www.worldometers.info/coronavirus/">Worldometers</a>
    </p>
    {table}
    <p>Regards,</p>
    <p>Covid-update</p>
    </body></html>
    """
    
    text = text.format(table=tabulate(data_dict.items(), tablefmt="grid"))
    html = html.format(table=tabulate(data_dict.items(), tablefmt="html"))
    
    message = MIMEMultipart(
        "alternative", None, [MIMEText(text), MIMEText(html,'html')])

    message['Subject'] = subject
    message['From'] = 'sender_email'
    message['To'] = 'receiver_email' # receiver

    
    server.sendmail(
        message['From'],
        message['To'],
        message.as_string()
    )
    print('Hey Email has been sent!')

    server.quit()

In [8]:
# sending email
send_mail_table(data_dict)


Hey Email has been sent!


## Important Links
* [COVID-19 Github Repo](https://github.com/ChaitanyaBaweja/COVID-19)
* [Worldometers Website](https://www.worldometers.info/coronavirus/)
* [Selenium Tutorial](https://dev.to/razgandeanu/selenium-cheat-sheet-9lc)
